In [1]:
!pip install google-auth gcsfs jupyterlab xarray zarr dask 

Defaulting to user installation because normal site-packages is not writeable
  Using cached google_auth-2.30.0-py2.py3-none-any.whl.metadata (4.7 kB)
  Using cached gcsfs-2024.6.0-py2.py3-none-any.whl.metadata (1.6 kB)
  Using cached xarray-2024.6.0-py3-none-any.whl.metadata (11 kB)
  Using cached zarr-2.18.2-py3-none-any.whl.metadata (5.7 kB)
  Using cached cachetools-5.3.3-py3-none-any.whl.metadata (5.3 kB)
  Using cached pyasn1_modules-0.4.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached fsspec-2024.6.0-py3-none-any.whl.metadata (11 kB)
  Using cached google_auth_oauthlib-1.2.0-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached google_cloud_storage-2.17.0-py2.py3-none-any.whl.metadata (6.6 kB)
  Using cached asciitree-0.3.3.tar.gz (4.0 kB)
  Preparing metadata (setup.py) ... done
  Using cached fasteners-0.19-py3-none-any.whl.metadata (4.9 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using ca

In [1]:
import gcsfs
import xarray as xr
from google.cloud import storage
from google.oauth2.credentials import Credentials

# import an access token
# - option 1: read an access token from a file
with open("token.txt") as f:
    access_token = f.read().strip()

# setup a storage client using credentials
credentials = Credentials(access_token)
fs = gcsfs.GCSFileSystem(token=credentials)

In [3]:
ds = xr.open_dataset(fs.get_mapper('gs://leap-scratch/jbusecke/ocean-emulators/test_CMIP6_GFDL-CM4.piControl.r1i1p1f1.zarr'), engine='zarr', chunks={})

In [4]:
ds

<xarray.Dataset> Size: 189GB
Dimensions:            (time: 6000, y: 154, x: 360, lev: 35)
Coordinates:
    lat                (y, x) float64 444kB dask.array<chunksize=(77, 360), meta=np.ndarray>
  * lev                (lev) float64 280B 2.5 10.0 20.0 ... 6e+03 6.5e+03
    lon                (y, x) float64 444kB dask.array<chunksize=(77, 360), meta=np.ndarray>
    sub_experiment_id  <U4 16B ...
  * time               (time) object 48kB 0151-01-16 12:00:00 ... 0650-12-16 ...
    variant_label      <U8 32B ...
  * x                  (x) float64 3kB 0.5 1.5 2.5 3.5 ... 357.5 358.5 359.5
  * y                  (y) float64 1kB -89.5 -88.5 -87.5 ... 61.5 62.5 63.5
Data variables:
    hft                (time, y, x) float32 1GB dask.array<chunksize=(1, 154, 360), meta=np.ndarray>
    so                 (time, lev, y, x) float32 47GB dask.array<chunksize=(1, 35, 154, 360), meta=np.ndarray>
    thetao             (time, lev, y, x) float32 47GB dask.array<chunksize=(1, 35, 154, 360), meta=np.ndarray>
    uo                 (time, lev, y, x) float32 47GB dask.array<chunksize=(1, 35, 154, 360), meta=np.ndarray>
    vo                 (time, lev, y, x) float32 47GB dask.array<chunksize=(1, 35, 154, 360), meta=np.ndarray>
    zos                (time, y, x) float32 1GB dask.array<chunksize=(1, 154, 360), meta=np.ndarray>
Attributes: (12/49)
    Conventions:                         CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:                         CMIP
    branch_method:                       standard
    branch_time_in_child:                0.0
    branch_time_in_parent:               54750.0
    comment:                             <null ref>
    ...                                  ...
    sub_experiment:                      none
    sub_experiment_id:                   none
    title:                               NOAA GFDL GFDL-CM4 model output prep...
    variant_info:                        N/A
    variant_label:                       r1i1p1f1
    version_id:                          v20180701

In [9]:
ds.isel(time=slice(0,3)).to_zarr('/pscratch/sd/s/shubhamg/test.zarr',
                                 encoding = {v:{'compressor':None} for v in ds.variables},
                                 consolidated=True, mode='w')

In [10]:
from dask.diagnostics import ProgressBar

In [12]:
with ProgressBar():
    ds.to_zarr('/pscratch/sd/s/shubhamg/test_CMIP6_GFDL-CM4.piControl.r1i1p1f1.zarr',
                                 encoding = {v:{'compressor':None} for v in ds.variables},
                                 consolidated=True, mode='w')

[########################################] | 100% Completed | 273.47 s
